In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Testing").getOrCreate()


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,377 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,129 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,762 kB in 2s (2,057 kB/s)
Reading package lists... Done


In [2]:
# Start Spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, when
spark = SparkSession.builder.appName("BankChurners").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://groupfourproject.s3.ca-central-1.amazonaws.com/bank_churners.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("bank_churners.csv"), header=True, sep=',', inferSchema=True)


# Show DataFrame
df.show()

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Naive_Bayes_Classifier_Attrit

In [4]:
#Showing Column Type
df.printSchema()

root
 |-- CLIENTNUM: integer (nullable = true)
 |-- Attrition_Flag: string (nullable = true)
 |-- Customer_Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Dependent_count: integer (nullable = true)
 |-- Education_Level: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Income_Category: string (nullable = true)
 |-- Card_Category: string (nullable = true)
 |-- Months_on_book: integer (nullable = true)
 |-- Total_Relationship_Count: integer (nullable = true)
 |-- Months_Inactive_12_mon: integer (nullable = true)
 |-- Contacts_Count_12_mon: integer (nullable = true)
 |-- Credit_Limit: double (nullable = true)
 |-- Total_Revolving_Bal: integer (nullable = true)
 |-- Avg_Open_To_Buy: double (nullable = true)
 |-- Total_Amt_Chng_Q4_Q1: double (nullable = true)
 |-- Total_Trans_Amt: integer (nullable = true)
 |-- Total_Trans_Ct: integer (nullable = true)
 |-- Total_Ct_Chng_Q4_Q1: double (nullable = true)
 |-- Avg_Utilization_Ratio: double (n

In [5]:
#Dropping duplicates within the dataframe
df.dropDuplicates().show()

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Naive_Bayes_Classifier_Attrit

In [6]:
#Drooping NA in the dataframe
df.na.drop(how='all').show()

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Naive_Bayes_Classifier_Attrit

In [7]:
#Showing the head of the dataframe
df.show(10)

#Displaying the tail of the dataframe
df.tail(10)
df.show()

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Naive_Bayes_Classifier_Attrit

In [8]:
df.drop('CLIENTNUM',
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2').show()


+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+--------

In [9]:
#Create a temporary view
df.createOrReplaceTempView("bank")


**Demographic Data to Analyze**

**Gender**

In [10]:
gender = spark.sql("""
SELECT
  Attrition_Flag as Flag,
  COUNT(Attrition_Flag) as Gender
FROM
  bank
WHERE
  Gender IN ('F', 'M')
GROUP BY
  Attrition_Flag
""")

gender.show()



+-----------------+------+
|             Flag|Gender|
+-----------------+------+
|Existing Customer|  8500|
|Attrited Customer|  1627|
+-----------------+------+



**Dependent Count**

In [11]:
dependent_count= spark.sql("""
SELECT
  Attrition_Flag as Flag,
  COUNT(*) as DependentCount,
  CASE
    WHEN Dependent_count BETWEEN 0 AND 5 THEN '0-5'
    ELSE 'Other'
  END AS DependentRange
FROM
  bank
GROUP BY
  Attrition_Flag,
  CASE
    WHEN Dependent_count BETWEEN 0 AND 5 THEN '0-5'
    ELSE 'Other'
  END
ORDER BY
  DependentRange
""")
dependent_count.show()

+-----------------+--------------+--------------+
|             Flag|DependentCount|DependentRange|
+-----------------+--------------+--------------+
|Attrited Customer|          1627|           0-5|
|Existing Customer|          8500|           0-5|
+-----------------+--------------+--------------+



**Education**

In [12]:
education = spark.sql("""
SELECT
  Attrition_Flag as Flag,
  Education_Level as Education,
  COUNT(Education_Level) as EducationCount
FROM
  bank
GROUP BY
  Attrition_Flag,
  Education_Level
ORDER BY
  Attrition_Flag,
  Education_Level
""")
education.show()

+-----------------+-------------+--------------+
|             Flag|    Education|EducationCount|
+-----------------+-------------+--------------+
|Attrited Customer|      College|           154|
|Attrited Customer|    Doctorate|            95|
|Attrited Customer|     Graduate|           487|
|Attrited Customer|  High School|           306|
|Attrited Customer|Post-Graduate|            92|
|Attrited Customer|   Uneducated|           237|
|Attrited Customer|      Unknown|           256|
|Existing Customer|      College|           859|
|Existing Customer|    Doctorate|           356|
|Existing Customer|     Graduate|          2641|
|Existing Customer|  High School|          1707|
|Existing Customer|Post-Graduate|           424|
|Existing Customer|   Uneducated|          1250|
|Existing Customer|      Unknown|          1263|
+-----------------+-------------+--------------+



**Income Category**

In [13]:
income = spark.sql("""
SELECT
  Attrition_Flag as Flag,
  Income_Category as Income,
  COUNT(*) as Income_Category,
  CASE
    WHEN Income_Category = 'Unknown' THEN 'Unknown'
    WHEN Income_Category <= 40000 THEN '<= 40,000'
    WHEN Income_Category BETWEEN 60000 and 80000 THEN '60,000-80,000'
    WHEN Income_Category BETWEEN 80000 and 120000 THEN '80,000-120,000'
    WHEN Income_Category > 120000 THEN '> 120,000'
    ELSE 'Other'
  END AS IncomeRange
FROM
  bank
GROUP BY
  Attrition_Flag,
   Income_Category,
  CASE
    WHEN Income_Category = 'Unknown' THEN 'Unknown'
    WHEN Income_Category <= 40000 THEN '<= 40,000'
    WHEN Income_Category BETWEEN 60000 and 80000 THEN '60,000-80,000'
    WHEN Income_Category BETWEEN 80000 and 120000 THEN '80,000-120,000'
    WHEN Income_Category > 120000 THEN '> 120,000'
    ELSE 'Other'
  END
ORDER BY
  Attrition_Flag
""")

income.show()


+-----------------+--------------+---------------+-----------+
|             Flag|        Income|Income_Category|IncomeRange|
+-----------------+--------------+---------------+-----------+
|Attrited Customer|   $40K - $60K|            271|      Other|
|Attrited Customer|       Unknown|            187|    Unknown|
|Attrited Customer|   $60K - $80K|            189|      Other|
|Attrited Customer|Less than $40K|            612|      Other|
|Attrited Customer|       $120K +|            126|      Other|
|Attrited Customer|  $80K - $120K|            242|      Other|
|Existing Customer|   $40K - $60K|           1519|      Other|
|Existing Customer|       $120K +|            601|      Other|
|Existing Customer|   $60K - $80K|           1213|      Other|
|Existing Customer|  $80K - $120K|           1293|      Other|
|Existing Customer|       Unknown|            925|    Unknown|
|Existing Customer|Less than $40K|           2949|      Other|
+-----------------+--------------+---------------+-----

**Behavioral (Bank) Data**

**Months on book**